In [ ]:
# create regression and classification label for the dataset
def prepare_dataset (df_in, period, df_truth_in = None, test_bool = bool):
    
    if not test_bool:    
        
        # make a dataframe to hold the last cycle for each engine in the dataset
        df_last_cycle = pd.DataFrame(df_in.groupby('id')['cycle'].max())
        df_last_cycle.reset_index(level=0, inplace=True)
        df_last_cycle.columns = ['id', 'last_cycle']

        # add time-to-failure ttf as a new column - regression label
        df_in = pd.merge(df_in, df_last_cycle, on='id')
        df_in['ttf'] = df_in['last_cycle'] - df_in['cycle']
        df_in.drop(['last_cycle'], axis=1, inplace=True)
    
        # create binary classification label
        df_in['label_bnc'] = df_in['ttf'].apply(lambda x: 1 if x <= period else 0)
    
        return df_in
    
    else:
        df_last_cycle = pd.DataFrame(df_in.groupby('id')['cycle'].max())
        df_last_cycle.reset_index(level=0, inplace=True)
        df_last_cycle.columns = ['id', 'last_cycle']
        df_truth_in["id"] = [i for i in range(1,101)]
        df_last_cycle = pd.merge(df_last_cycle, df_truth_in, on='id')
        df_last_cycle["last_cycle"] = df_last_cycle["last_cycle"] + df_last_cycle["ttf"]
        df_last_cycle.drop(['ttf'], axis=1, inplace=True)
        df_last_cycle.reset_index(drop=True, inplace=True)
        
        df_in = pd.merge(df_in, df_last_cycle, on='id')
        df_in['ttf'] = df_in['last_cycle'] - df_in['cycle']
        df_in.drop(['last_cycle'], axis=1, inplace=True)

        # create binary classification label
        df_in['label_bnc'] = df_in['ttf'].apply(lambda x: 1 if x <= period else 0)

        return df_in